<a href="https://colab.research.google.com/github/SaurabhKukreja/30Days_Prep/blob/master/LearnSalesClaimsData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 61.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=1730bd58c8c11fd28f152b78c05bb96f39738f8ad4e4525d977ea03ff02184b5
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from google.colab import drive
from pyspark.sql import SparkSession
import pandas as pd

In [ ]:
spark = (
    SparkSession.builder.appName("app_name").getOrCreate()
)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pyspark
from pyspark.sql.functions import to_date, col, sum

In [ ]:
enrollment_file = '/content/drive/My Drive/Colab Notebooks/Arine/enrollment.csv'
pharmacy_claim = '/content/drive/My Drive/Colab Notebooks/Arine/pharmacy_claims/*'
lookup = '/content/drive/My Drive/Colab Notebooks/Arine/ndc9_lookup.json'
enrol_df = spark.read.option("header",True).csv(enrollment_file)
claims_df = spark.read.option("header",True).csv(pharmacy_claim)
lookup_df = spark.read.option("header",True).csv(lookup)

# Evaluating Enrollment File

In [ ]:
enrol_df.count()

4597

In [ ]:
enrol_df.show()
enrol_df.printSchema()


+----------+-------------+-------------------+---------------------+----------+------+-----------+
|birth_date|      card_id|enrollment_end_date|enrollment_start_date|first_name|gender|  last_name|
+----------+-------------+-------------------+---------------------+----------+------+-----------+
|1970-12-13|ID00032868314|         2018-11-01|           2018-05-01|      Saul|     M|     Nguyen|
|1970-12-13|ID00032868314|         2999-12-31|           2019-08-01|      Saul|     M|     Nguyen|
|1976-09-15|ID00039326267|         2020-04-28|           2020-06-01|      Mina|     F|      Jones|
|1954-07-18|ID00065722935|         2020-05-28|           2018-05-01|   Barbara|     F|       Webb|
|1967-09-17|ID00077400893|         2020-07-28|           2019-01-01|    Carmen|     F|  Mcclendon|
|1953-02-12|ID00081368696|         2999-12-31|           2020-01-31|     David|     M|Rittenhouse|
|1972-07-17|ID00118584072|         2999-12-31|           2019-04-01|     James|     M|    Pearson|
|1965-11-0

**First I am checking if there are any nulls in enrollment_end_date to make sure every patient has an enrollment end date.**

In [ ]:
enrol_df.where(col(("enrollment_end_date")).isNull()).count()

0

Question 1 : How many patients were enrolled in the program as of July 1st, 2020? 
Answer : 2546

Assumption : When we say " as of July 1st 2020", I am understanding it that patients whose end_date is less than or equal to July 1st 2020.

In [ ]:
enrol_df.filter("enrollment_end_date <= date'2020-07-01'").count()

2546

# Pharmacy Claims

Note: the columns: 'card_id', 'claim_number', 'fill_date', 'ndc_11' are sufficient to identify an unique fill.

In [ ]:
claims_df.show()

+--------------+-----------+-----------------+------------+-----------+---------+---------------------+--------------------+-----------+----------+-----------+-------------+------------+---------------+-------------+--------------+------------------+-------------+----------------+----------+-------+-------------------+--------------+--------------+----------+--------------------+
|allowed_amount|    card_id|claim_line_number|claim_status|days_supply|fill_date|maintenance_drug_flag|     medication_name|     ndc_11| paid_date|pharmacy_id|pharmacy_name|pharmacy_npi|pharmacy_tax_id|prescriber_id|prescriber_npi|quantity_dispensed|refill_number|retail_mail_flag|  run_date| rxtype|specialty_drug_flag|strength_units|strength_value|uploadDate|        claim_number|
+--------------+-----------+-----------------+------------+-----------+---------+---------------------+--------------------+-----------+----------+-----------+-------------+------------+---------------+-------------+--------------+---

**Question 2 : how many rows are there in the initial pharmacy claims data set?**
Answer : 11524

In [ ]:
claims_df.count()

11524

In [ ]:
valid_claims = claims_df.filter(claims_df.fill_date.isNotNull()).filter(claims_df.claim_status!='DENIED')

In [ ]:
valid_claims.show()

+--------------+-----------+-----------------+------------+-----------+----------+---------------------+--------------------+-----------+----------+-----------+-------------+------------+---------------+-------------+--------------+------------------+-------------+----------------+----------+-------+-------------------+--------------+--------------+----------+--------------------+
|allowed_amount|    card_id|claim_line_number|claim_status|days_supply| fill_date|maintenance_drug_flag|     medication_name|     ndc_11| paid_date|pharmacy_id|pharmacy_name|pharmacy_npi|pharmacy_tax_id|prescriber_id|prescriber_npi|quantity_dispensed|refill_number|retail_mail_flag|  run_date| rxtype|specialty_drug_flag|strength_units|strength_value|uploadDate|        claim_number|
+--------------+-----------+-----------------+------------+-----------+----------+---------------------+--------------------+-----------+----------+-----------+-------------+------------+---------------+-------------+--------------+

In [ ]:
df_1 = valid_claims.groupBy('card_id','claim_number','fill_date','ndc_11').agg(sum('days_supply').alias('net_days_supply'), sum('allowed_amount').alias('net_allowed_amount'))

In [ ]:
valid_days_df = df_1.filter(df_1.net_days_supply > 0)

In [ ]:
valid_days_df.count()

3586

**Question 3: How many prepared claims do you have at the end of step 3?
Answer : 3586**

In [114]:
valid_days_df.show()

+-----------+--------------------+----------+-----------+---------------+------------------+
|    card_id|        claim_number| fill_date|     ndc_11|net_days_supply|net_allowed_amount|
+-----------+--------------------+----------+-----------+---------------+------------------+
|14352121939|80210894373363104910|2018-08-01|69097014912|           40.0|             11.44|
|85027626890|92648304520199540092|2019-11-01|11534018701|            5.0|             62.61|
|60216351600|77749091582381951081|2018-05-01|16729018201|           30.0|              5.46|
|36614641190|90288769027692286047|2018-02-01|65162010250|           30.0|              2.59|
|41403827121|45593195524218760836|2019-10-01|00378718705|           30.0|              5.89|
|57039941566|87434987409725815380|2019-09-01|00378718705|           90.0|             16.97|
|37307523917|72516389673490944291|2019-05-01|45802006405|           30.0|             17.28|
|40590474120|29763401176651419802|2017-08-01|00781523864|            3

In [115]:
valid_claims.show()

+--------------+-----------+-----------------+------------+-----------+----------+---------------------+--------------------+-----------+----------+-----------+-------------+------------+---------------+-------------+--------------+------------------+-------------+----------------+----------+-------+-------------------+--------------+--------------+----------+--------------------+
|allowed_amount|    card_id|claim_line_number|claim_status|days_supply| fill_date|maintenance_drug_flag|     medication_name|     ndc_11| paid_date|pharmacy_id|pharmacy_name|pharmacy_npi|pharmacy_tax_id|prescriber_id|prescriber_npi|quantity_dispensed|refill_number|retail_mail_flag|  run_date| rxtype|specialty_drug_flag|strength_units|strength_value|uploadDate|        claim_number|
+--------------+-----------+-----------------+------------+-----------+----------+---------------------+--------------------+-----------+----------+-----------+-------------+------------+---------------+-------------+--------------+

In [128]:
data_joined = valid_days_df.join(valid_claims, (valid_days_df.card_id == valid_claims.card_id) & 
                                 (valid_days_df.claim_number==valid_claims.claim_number)&
                                 (valid_days_df.fill_date== valid_claims.fill_date)&
                                 (valid_days_df.ndc_11 ==valid_claims.ndc_11)).dropDuplicates()


In [146]:
data_joined = valid_days_df.join(valid_claims, ["card_id","claim_number","fill_date","ndc_11"]).dropDuplicates()

In [147]:
data_joined.printSchema()

root
 |-- card_id: string (nullable = true)
 |-- claim_number: string (nullable = true)
 |-- fill_date: string (nullable = true)
 |-- ndc_11: string (nullable = true)
 |-- net_days_supply: double (nullable = true)
 |-- net_allowed_amount: double (nullable = true)
 |-- allowed_amount: string (nullable = true)
 |-- claim_line_number: string (nullable = true)
 |-- claim_status: string (nullable = true)
 |-- days_supply: string (nullable = true)
 |-- maintenance_drug_flag: string (nullable = true)
 |-- medication_name: string (nullable = true)
 |-- paid_date: string (nullable = true)
 |-- pharmacy_id: string (nullable = true)
 |-- pharmacy_name: string (nullable = true)
 |-- pharmacy_npi: string (nullable = true)
 |-- pharmacy_tax_id: string (nullable = true)
 |-- prescriber_id: string (nullable = true)
 |-- prescriber_npi: string (nullable = true)
 |-- quantity_dispensed: string (nullable = true)
 |-- refill_number: string (nullable = true)
 |-- retail_mail_flag: string (nullable = true)


In [150]:
claims_enrollment_df = data_joined.join(enrol_df, enrol_df.card_id==data_joined.card_id).show()

+-------+------------+---------+------+---------------+------------------+--------------+-----------------+------------+-----------+---------------------+---------------+---------+-----------+-------------+------------+---------------+-------------+--------------+------------------+-------------+----------------+--------+------+-------------------+--------------+--------------+----------+----------+-------+-------------------+---------------------+----------+------+---------+
|card_id|claim_number|fill_date|ndc_11|net_days_supply|net_allowed_amount|allowed_amount|claim_line_number|claim_status|days_supply|maintenance_drug_flag|medication_name|paid_date|pharmacy_id|pharmacy_name|pharmacy_npi|pharmacy_tax_id|prescriber_id|prescriber_npi|quantity_dispensed|refill_number|retail_mail_flag|run_date|rxtype|specialty_drug_flag|strength_units|strength_value|uploadDate|birth_date|card_id|enrollment_end_date|enrollment_start_date|first_name|gender|last_name|
+-------+------------+---------+------

In [125]:
df_test=data_joined.dropDuplicates()

In [126]:
df_test.count()

4005

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data2 = [("1","1234","PAID",100),
    ("1","1234","DECLINED",-100),
    ("2","123456","PAID",100)
  ]

schema = StructType([ \
    StructField("ID",StringType(),True), \
    StructField("Number",StringType(),True), \
    StructField("Status",StringType(),True),
    StructField("Amount",IntegerType(),True)
  ])
 
df = spark.createDataFrame(data=data2,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- ID: string (nullable = true)
 |-- Number: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Amount: integer (nullable = true)

+---+------+--------+------+
|ID |Number|Status  |Amount|
+---+------+--------+------+
|1  |1234  |PAID    |100   |
|1  |1234  |DECLINED|-100  |
|2  |123456|PAID    |100   |
+---+------+--------+------+



In [ ]:
df_1 = df.groupBy('Number', 'ID').agg(sum('Amount').alias('sum'))

In [ ]:
df_1.filter(df_1.sum > 0).show()

+------+---+---+
|Number| ID|sum|
+------+---+---+
|123456|  2|100|
+------+---+---+

